# Pertinent Nomad

#### Initial Goal
 

In [940]:
import sys
from typing import List, Tuple, Dict
# TODO install module rather than inserting to path
sys.path.insert(0, '..')
import os
import math
import time
import pandas as pd
import numpy as np
import gmaps
import folium
import itertools as it
import great_circle_calculator.great_circle_calculator as gcc

from common.helpers import nomad as nom
import logging

def myLogger():
  logger = logging.getLogger(__name__)
  formatter = logging.Formatter('%(asctime)s: %(lineno)2s: %(levelname)s: %(message)s')
  logger.setLevel(logging.DEBUG)
  logger.propagate = False
  filename = 'nomad_intersections.log'
  if (logger.hasHandlers()):
    logger.handlers.clear()

  fhandler = logging.FileHandler(filename=filename, mode='a')
  fhandler.setFormatter(formatter)
  fhandler.setLevel(logging.ERROR)
  logger.addHandler(fhandler)

  consoleHandler = logging.StreamHandler(sys.stdout)
  consoleHandler.setFormatter(formatter)
  consoleHandler.setLevel(logging.DEBUG)
  logger.addHandler(consoleHandler)

  return logger

logger = myLogger()

# Read CSVs into Panda DataFrames
# TODO move csvs to external location
# TODO read one combined csv file instead of reading folder
csv_path = '../common/csv/single/'
csv_list = os.listdir(csv_path)
logger.debug(f'{len(csv_list)} .csv files found.')
logger.debug(csv_list)

# Dictionary with DataFrames
# Coordinates are initally in Lat-Lon
stores_latlon = {}
for file in csv_list:
  short_name = file.split('_', 1)[0]
  stores_latlon[short_name] = pd.read_csv('../common/csv/single/' + file, usecols=['Latitude', 'Longitude'])


2022-10-17 20:00:24,387: 46: DEBUG: 3 .csv files found.
2022-10-17 20:00:24,387: 47: DEBUG: ['container_locations_clean.csv', 'ikea_locations_clean.csv', 'micro_locations_clean.csv']


In [941]:
stores_dataframe = pd.read_csv('../common/csv/merged_locations_clean.csv', dtype={'store_zip': 'str'})

logger.debug(f'Unique short_name found: {stores_dataframe.short_name.unique()}')

stores_dataframe.head(10)


2022-10-17 20:00:24,432:  3: DEBUG: Unique short_name found: ['containerstore' 'ikea' 'microcenter']


,store_uid,uuid,short_name,store_name,store_desc,store_add1,store_add2,store_city,store_state,store_zip,store_address_full,Latitude,Longitude,rand_uuid
0,01960_8e51e0a5_containerstore_peabody,8e51e0a5,containerstore,Container Store,Peabody,210 Andover Street,LL01,Peabody,MA,01960,"210 Andover Street Peabody, MA 01960",42.541420,-70.944590,40eaa5d2
1,02467_1a370ad1_containerstore_chestnut_hill,1a370ad1,containerstore,Container Store,Chestnut Hill,55 Boylston St,NaN,Chestnut Hill,MA,02467,"55 Boylston St Chestnut Hill, MA 02467",42.322783,-71.168710,ebe56afc
2,01760_bd4ac9aa_containerstore_natick,bd4ac9aa,containerstore,Container Store,Natick,1265 Worcester Street,NaN,Natick,MA,01760,"1265 Worcester Street Natick, MA 01760",42.299782,-71.389396,445fe8ff
3,02920_0d186a8a_containerstore_cranston,0d186a8a,containerstore,Container Store,Cranston,150 Hillside Rd,NaN,Cranston,RI,02920,"150 Hillside Rd Cranston, RI 02920",41.760420,-71.457487,8fe60c8b
4,11530_a0fa003f_containerstore_garden_city,a0fa003f,containerstore,Container Store,Garden City,902 Old Country Rd,NaN,Garden City,NY,11530,"902 Old Country Rd Garden City, NY 11530",40.744808,-73.603854,6ecb7029
5,10601_fdd1e407_containerstore_white_plains,fdd1e407,containerstore,Container Store,White Plains,145 Westchester Avenue,NaN,White Plains,NY,10601,"145 Westchester Avenue White Plains, NY 10601",41.032201,-73.757623,80e926e9
6,10710_1d7c8925_containerstore_yonkers,1d7c8925,containerstore,Container Store,Yonkers,48 Market Street,NaN,Yonkers,NY,10710,"48 Market Street Yonkers, NY 10710",40.961632,-73.855598,28c81e6b
7,10011_0ca8d163_containerstore_new_york,0ca8d163,containerstore,Container Store,New York,629 Sixth Avenue,NaN,New York,NY,10011,"629 Sixth Avenue New York, NY 10011",40.740346,-73.995217,9c8a3b74
8,07652_91b00fdd_containerstore_paramus,91b00fdd,containerstore,Container Store,Paramus,370 Route 17,#1,Paramus,NJ,07652,"370 Route 17 Paramus, NJ 07652",40.945984,-74.071044,51c16766
9,10314_d3b6df1b_containerstore_staten_island,d3b6df1b,containerstore,Container Store,Staten Island,283 Platinum Ave,Suite 1365,Staten Island,NY,10314,"283 Platinum Ave Staten Island, NY 10314",40.580122,-74.166750,8bb1f374


In [942]:
# Earth radius in miles
earth_r = 3958.8

# Eeach store needs a radius to check (miles)
ikea_r = 100
micro_r = 25

# Names should match csv_list
stores_radii = {
  'ikea': 50,
  'micro': 30,
  'container': 20
}
stores_radii_ = {
  'ikea': 50,
  'microcenter': 30,
  'containerstore': 20
}

In [943]:
stores_dataframe['coordinates'] = list(zip(stores_dataframe.Longitude, stores_dataframe.Latitude))
# stores_dataframe.head()

stores_dataframe_c = stores_dataframe.copy()

stores_dataframe_c = stores_dataframe[['short_name', 'coordinates']].groupby('short_name').agg({'coordinates': lambda c: list(c)}).reset_index()
stores_dataframe_c = stores_dataframe_c.rename(columns={'short_name': 'store'})
stores_dataframe_c['radius'] = stores_dataframe_c['store'].map(stores_radii_)
stores_dataframe_c = stores_dataframe_c.sort_values(by='radius', ascending=False, ignore_index=True)

print(stores_dataframe_c.iloc[0:])
stores_dataframe_c.head()

            store                                        coordinates  radius
0            ikea  [(-71.0684562, 42.1371268), (-72.9200335, 41.2...      50
1     microcenter  [(-71.1143075, 42.3572095), (-73.5858212, 40.7...      30
2  containerstore  [(-70.9445901, 42.5414196), (-71.1687096, 42.3...      20


,store,coordinates,radius
0,ikea,"[(-71.0684562, 42.1371268), (-72.9200335, 41.2...",50
1,microcenter,"[(-71.1143075, 42.3572095), (-73.5858212, 40.7...",30
2,containerstore,"[(-70.9445901, 42.5414196), (-71.1687096, 42.3...",20


In [944]:
stores_dataframe_c.head(5)


,store,coordinates,radius
0,ikea,"[(-71.0684562, 42.1371268), (-72.9200335, 41.2...",50
1,microcenter,"[(-71.1143075, 42.3572095), (-73.5858212, 40.7...",30
2,containerstore,"[(-70.9445901, 42.5414196), (-71.1687096, 42.3...",20


In [957]:
stores_dataframe_d = stores_dataframe_c.copy()
stores_dataframe_d = stores_dataframe_d.to_dict('records')

print(stores_dataframe_d)

[{'store': 'ikea', 'coordinates': [(-71.0684562, 42.1371268), (-72.9200335, 41.2958017), (-73.5310569, 40.7748383), (-73.8618461, 40.731424), (-74.0115416, 40.672219), (-74.0732861, 40.9245349), (-74.1697393, 40.6751301), (-75.1417686, 39.9171256), (-75.3067793, 40.0943896), (-76.2013761, 36.8760812), (-76.4619125, 39.3753517), (-76.928835, 39.0206756), (-77.2898272, 38.6450421), (-80.1681502, 40.4517367), (-80.33035, 26.122275), (-80.3836105, 25.7914278), (-80.7650357, 35.29331), (-81.4266415, 28.4829579), (-81.5167824, 30.2374622), (-82.4316019, 27.9542754), (-82.9667899, 40.1501367), (-83.4513499, 42.3252624), (-84.4051193, 33.7888368), (-84.4339977, 39.3154678), (-86.0073736, 39.9534162), (-87.9386886, 42.9076915), (-88.0365045, 42.0566646), (-88.037837, 41.7280902), (-89.7986837, 35.1902921), (-90.2449962, 38.6335827), (-93.2448492, 44.8586403), (-94.6905789, 39.019658), (-95.4715555, 29.7855758), (-96.8213668, 33.0937506), (-97.022627, 32.6906294), (-97.6897453, 30.5578282), (-98

In [945]:
# Dictionary with Lon-Lat DataFrames
# Convert to Lon-Lat format beacuse the library accepts points in lon-lat
stores = {}
def dfs_to_lonlat(dfs):
  for store in dfs:
    stores[store] = dfs[store][['Longitude', 'Latitude']]

dfs_to_lonlat(stores_latlon)

# print('1.stores: ', stores)

for key in stores:
  print(f'DataFrame[Lon, Lat]: stores["{key}"]')
  # stores[key].head()

# TODO: change name of _np since not converting to numpy arrays anymore
# Creating Dict with numpy arrays instead of DataFrames
stores_np = {}
for key in stores:
  # stores_np[key] = stores[key].to_numpy()
  stores_np[key] = list(stores[key].itertuples(index=False, name=None))
  # print(list(stores_np[key].itertuples(index=False, name=None)))

for store_np in stores_np:
  print(f'numpy[Lon, Lat]: stores_np["{store_np}"]')
  print(stores_np[store_np][:5])

ikea_df = pd.read_csv('../common/csv/single/ikea_locations_clean.csv', usecols=['Longitude', 'Latitude'])
micro_df = pd.read_csv('../common/csv/single/micro_locations_clean.csv', usecols=['Longitude', 'Latitude'])
cont_df = pd.read_csv('../common/csv/single/container_locations_clean.csv', usecols=['Longitude', 'Latitude'])

ikea_df_lonlat = ikea_df[['Longitude', 'Latitude']]

DataFrame[Lon, Lat]: stores["container"]
DataFrame[Lon, Lat]: stores["ikea"]
DataFrame[Lon, Lat]: stores["micro"]
numpy[Lon, Lat]: stores_np["container"]
[(-112.2223196, 33.6398132), (-77.6150483, 37.6536924), (-77.3549644, 38.9616959), (-77.092154, 38.8875055), (-111.8878794, 40.6361941)]
numpy[Lon, Lat]: stores_np["ikea"]
[(-83.4513499, 42.3252624), (-93.2448492, 44.8586403), (-80.7650357, 35.29331), (-90.2449962, 38.6335827), (-76.4619125, 39.3753517)]
numpy[Lon, Lat]: stores_np["micro"]
[(-117.8328829, 33.7245358), (-77.2615284, 38.8689425), (-84.4298998, 39.2882802), (-96.7503193, 32.9374633), (-81.4373282, 41.5239851)]


In [955]:
# Prepare stores dictionaries for iteration with added radius
stores_np_rad = []
for store in stores_np:
  stores_np_rad.append({'store': store, 'radius': stores_radii[store], 'points': stores_np[store]})

# Sort np_rad dict by radius descending
# stores_np_rad = {key: value for key, value in sorted(stores_np_rad.items(), key=lambda item: item[1]['radius'], reverse=True)}
stores_np_rad = sorted(stores_np_rad, key=lambda d: d['radius'], reverse=True)

# Accessing the list of points in lon-lat
print(stores_np_rad)

[{'store': 'ikea', 'radius': 50, 'points': [(-83.4513499, 42.3252624), (-93.2448492, 44.8586403), (-80.7650357, 35.29331), (-90.2449962, 38.6335827), (-76.4619125, 39.3753517), (-76.928835, 39.0206756), (-71.0684562, 42.1371268), (-94.6905789, 39.019658), (-88.037837, 41.7280902), (-88.0365045, 42.0566646), (-84.4051193, 33.7888368), (-86.0073736, 39.9534162), (-82.4316019, 27.9542754), (-80.33035, 26.122275), (-80.3836105, 25.7914278), (-87.9386886, 42.9076915), (-111.8931892, 40.5086171), (-97.6897453, 30.5578282), (-122.2280158, 47.4426763), (-95.4715555, 29.7855758), (-77.2898272, 38.6450421), (-98.3381062, 29.5665157), (-81.4266415, 28.4829579), (-96.8213668, 33.0937506), (-76.2013761, 36.8760812), (-80.1681502, 40.4517367), (-97.022627, 32.6906294), (-89.7986837, 35.1902921), (-122.5535877, 45.5712001), (-82.9667899, 40.1501367), (-74.0115416, 40.672219), (-73.5310569, 40.7748383), (-75.1417686, 39.9171256), (-115.2806099, 36.0688447), (-75.3067793, 40.0943896), (-74.1697393, 40.

In [947]:
stores_np_rad_df = pd.DataFrame(stores_np_rad)
stores_np_rad_df

,store,radius,points
0,ikea,50,"[(-83.4513499, 42.3252624), (-93.2448492, 44.8..."
1,micro,30,"[(-117.8328829, 33.7245358), (-77.2615284, 38...."
2,container,20,"[(-112.2223196, 33.6398132), (-77.6150483, 37...."


In [948]:
def int_cluster_points_multi(stores: list):
  
  anchor = stores.iloc[0]
  target = stores.iloc[1]
  targets = stores.iloc[1:]
  anchor_radius = anchor['radius']
  targets_count = len(targets)

  logger.debug(f'Using Store: ({len(anchor["coordinates"])}) {anchor["store"].upper()} as Anchor Points with Radius: {anchor["radius"]}')
  logger.debug(f'Additional Stores: {targets_count}')
  logger.debug(f'First Target: ({len(target["coordinates"])}) {target["store"].upper()} with Radius: {target["radius"]} ')

  local_points = []
  intermediate_points = []

  for anchor_point in anchor['coordinates']:

    target_point_clusters: List[Tuple(float, float)] = []
    intermediate_point_clusters: List[Tuple(float, float)] = []

    for i in range(1, targets_count):
      target_radius = target['radius']

      for target_point in stores.iloc[i]['coordinates']:
        if i < targets_count:
          target_intersections = []
          anchor_distance = gcc.distance_between_points(anchor_point, target_point, unit='miles')

          if anchor_distance < anchor_radius + target_radius:
            anchor_target_inter = gcc.intermediate_point(anchor_point, target_point, nom._intermediate_ratio(anchor_radius, target_radius))

            target_point_clusters[0:] = [(anchor_point[1], anchor_point[0])]
            intermediate_point_clusters[0:] = [(anchor_target_inter[1], anchor_target_inter[0])]

            next_radius = stores.iloc[i + 1]['radius']

            for next_target in stores.iloc[i + 1]['coordinates']:
              
              target_next_distance = gcc.distance_between_points(target_point, next_target, unit='miles')
              
              if target_next_distance < target_radius + next_radius:
                inter_target_next = gcc.intermediate_point(target_point, next_target, nom._intermediate_ratio(target_radius, next_radius))
                intermediate_point_clusters += [(inter_target_next[1], inter_target_next[0])]

                target_point_clusters[i:] = [(target_point[1], target_point[0])]
                target_intersections += [(next_target[1], next_target[0])]

            if len(target_intersections) > 0:
              target_point_clusters += target_intersections
            

        if i == targets_count:
          break
    
    if len(target_point_clusters) > 0:
      local_points += [target_point_clusters]
    
    if len(intermediate_point_clusters) > 0:
      intermediate_points += [intermediate_point_clusters]

  return intermediate_points, local_points

t0 = time.perf_counter()
intermediate_points, local_points = int_cluster_points_multi(stores_dataframe_c)
t1 = time.perf_counter()

logger.debug(f'int_cluster_points_multi took {round((t1-t0) * 1000, 2)} ms and found {len(intermediate_points)} clusters.')
# print(intermediate_points[:5])

intermediate_points_dataframe = pd.DataFrame(intermediate_points)
intermediate_points_dataframe.head(10)

2022-10-17 20:00:25,093:  9: DEBUG: Using Store: (51) IKEA as Anchor Points with Radius: 50
2022-10-17 20:00:25,095: 10: DEBUG: Additional Stores: 2
2022-10-17 20:00:25,098: 11: DEBUG: First Target: (25) MICROCENTER with Radius: 30 
2022-10-17 20:00:25,331: 67: DEBUG: int_cluster_points_multi took 238.56 ms and found 31 clusters.


,0,1,2,3,4,5,6,7,8
0,"(42.274680628854114, -71.09707574838758)","(42.46776561630177, -71.0125968909876)","(42.33655674568646, -71.14695590809627)","(42.3228315907223, -71.27942064884141)","(41.99925843622335, -71.3209880309333)",None,None,None,None
1,"(41.05420842419934, -73.67998660801094)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
2,"(40.85772446263708, -73.90715492677926)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
3,"(40.841151570920545, -74.03131354270296)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
4,"(40.81889012301009, -74.08751678714235)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
5,"(40.91349722875724, -74.1107756066327)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
6,"(40.27969787266764, -74.9222249688945)","(39.981666224821865, -75.1630168539542)","(40.068948394371596, -75.37771549648211)","(39.82050072153811, -75.53792942643233)",None,None,None,None,None
7,"(39.99404820821346, -75.28399383511584)","(39.981666224821865, -75.1630168539542)","(40.068948394371596, -75.37771549648211)","(39.82050072153811, -75.53792942643233)",None,None,None,None,None
8,"(40.06047393574766, -75.34598063203947)","(39.981666224821865, -75.1630168539542)","(40.068948394371596, -75.37771549648211)","(39.82050072153811, -75.53792942643233)",None,None,None,None,None
9,"(39.059485239503545, -76.96301845596422)","(38.94274694870166, -76.8321660275575)","(38.916662445915705, -77.15264484786259)","(38.880109659550804, -77.15991437432032)","(38.98571566057485, -77.1805969972741)","(38.90235403775053, -77.24857030605422)","(38.924603478657744, -77.31756068678723)",None,None


In [959]:
def int_cluster_points_multi(stores):
  
  anchor = stores[0]
  target = stores[1]
  targets = stores[1:]
  anchor_radius = anchor['radius']
  targets_count = len(targets)

  logger.debug(f'Using Store: ({len(anchor["coordinates"])}) {anchor["store"].upper()} as Anchor Points with Radius: {anchor["radius"]}')
  logger.debug(f'Additional Stores: {targets_count}')
  logger.debug(f'First Target: ({len(target["coordinates"])}) {target["store"].upper()} with Radius: {target["radius"]} ')

  local_points = []
  intermediate_points = []

  for anchor_point in stores[0]['coordinates']:

    target_point_clusters: List[Tuple(float, float)] = []
    intermediate_point_clusters: List[Tuple(float, float)] = []

    for i in range(1, targets_count):
      target_radius = target['radius']

      for target_point in stores[i]['coordinates']:
        if i < targets_count:
          target_intersections = []
          anchor_distance = gcc.distance_between_points(anchor_point, target_point, unit='miles')

          if anchor_distance < anchor_radius + target_radius:
            anchor_target_inter = gcc.intermediate_point(anchor_point, target_point, nom._intermediate_ratio(anchor_radius, target_radius))

            target_point_clusters[0:] = [(anchor_point[1], anchor_point[0])]
            intermediate_point_clusters[0:] = [(anchor_target_inter[1], anchor_target_inter[0])]

            next_radius = stores[i + 1]['radius']

            for next_target in stores[i + 1]['coordinates']:
              
              target_next_distance = gcc.distance_between_points(target_point, next_target, unit='miles')
              
              if target_next_distance < target_radius + next_radius:
                inter_target_next = gcc.intermediate_point(target_point, next_target, nom._intermediate_ratio(target_radius, next_radius))
                intermediate_point_clusters += [(inter_target_next[1], inter_target_next[0])]

                target_point_clusters[i:] = [(target_point[1], target_point[0])]
                target_intersections += [(next_target[1], next_target[0])]

            if len(target_intersections) > 0:
              target_point_clusters += target_intersections
            

        if i == targets_count:
          break
    
    if len(target_point_clusters) > 0:
      local_points += [target_point_clusters]
    
    if len(intermediate_point_clusters) > 0:
      intermediate_points += [intermediate_point_clusters]

  return intermediate_points, local_points

t0 = time.perf_counter()
intermediate_points, local_points = int_cluster_points_multi(stores_dataframe_d)
t1 = time.perf_counter()

logger.debug(f'int_cluster_points_multi took {round((t1-t0) * 1000, 2)} ms and found {len(intermediate_points)} clusters.')
# print(intermediate_points[:5])

intermediate_points_dataframe = pd.DataFrame(intermediate_points)
intermediate_points_dataframe.head(10)

2022-10-17 20:08:35,355:  9: DEBUG: Using Store: (51) IKEA as Anchor Points with Radius: 50
2022-10-17 20:08:35,357: 10: DEBUG: Additional Stores: 2
2022-10-17 20:08:35,361: 11: DEBUG: First Target: (25) MICROCENTER with Radius: 30 
2022-10-17 20:08:35,514: 67: DEBUG: int_cluster_points_multi took 159.22 ms and found 31 clusters.


,0,1,2,3,4,5,6,7,8
0,"(42.274680628854114, -71.09707574838758)","(42.46776561630177, -71.0125968909876)","(42.33655674568646, -71.14695590809627)","(42.3228315907223, -71.27942064884141)","(41.99925843622335, -71.3209880309333)",None,None,None,None
1,"(41.05420842419934, -73.67998660801094)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
2,"(40.85772446263708, -73.90715492677926)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
3,"(40.841151570920545, -74.03131354270296)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
4,"(40.81889012301009, -74.08751678714235)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
5,"(40.91349722875724, -74.1107756066327)","(40.80992237036381, -73.81530729781629)","(40.98221458605286, -73.90805011834583)","(40.939806894830205, -73.9667192849604)","(40.806974917245796, -74.05035254191289)","(40.93034187336925, -74.09594061931874)","(40.710821640541, -74.15339501520525)","(40.84097139670313, -74.25772131314014)","(40.70921183300416, -74.39039238028757)"
6,"(40.27969787266764, -74.9222249688945)","(39.981666224821865, -75.1630168539542)","(40.068948394371596, -75.37771549648211)","(39.82050072153811, -75.53792942643233)",None,None,None,None,None
7,"(39.99404820821346, -75.28399383511584)","(39.981666224821865, -75.1630168539542)","(40.068948394371596, -75.37771549648211)","(39.82050072153811, -75.53792942643233)",None,None,None,None,None
8,"(40.06047393574766, -75.34598063203947)","(39.981666224821865, -75.1630168539542)","(40.068948394371596, -75.37771549648211)","(39.82050072153811, -75.53792942643233)",None,None,None,None,None
9,"(39.059485239503545, -76.96301845596422)","(38.94274694870166, -76.8321660275575)","(38.916662445915705, -77.15264484786259)","(38.880109659550804, -77.15991437432032)","(38.98571566057485, -77.1805969972741)","(38.90235403775053, -77.24857030605422)","(38.924603478657744, -77.31756068678723)",None,None


In [962]:
def centroid_points_cluster(cluster: List[List[(Tuple[float, float])]]):
  cent_points: List[Tuple(float, float)] = []

  for region in cluster:
    # print(region)
    cent_point = nom._cent_points(region)
    cent_points += [cent_point]

  return cent_points

centroid_points = centroid_points_cluster(intermediate_points)

centroid_points_dataframe = pd.DataFrame(centroid_points, columns=['Latitude', 'Longitude'])
# centroid_points_dataframe.to_clipboard()
logger.debug(f'({len(centroid_points_dataframe.index)}) Points Found')
centroid_points_dataframe.head()

2022-10-17 20:10:10,341: 15: DEBUG: (31) Points Found


,Latitude,Longitude
0,42.280276,-71.171624
1,40.865128,-74.035572
2,40.843241,-74.060690
3,40.841388,-74.074481
4,40.838914,-74.080726


In [963]:
local_points = centroid_points_cluster(local_points)

local_points_dataframe = pd.DataFrame(local_points, columns=['Latitude', 'Longitude'])
# local_points_dataframe.to_clipboard()
logger.debug(f'({len(local_points_dataframe.index)}) Points Found')
local_points_dataframe.head()

2022-10-17 20:10:15,264:  5: DEBUG: (31) Points Found


,Latitude,Longitude
0,42.236597,-71.190988
1,40.858981,-73.941548
2,40.806494,-74.001869
3,40.802061,-74.034952
4,40.796127,-74.049935


In [964]:

ikea_latlon_df = ikea_df[['Longitude', 'Latitude']].to_numpy()
micro_latlon_df = micro_df[['Longitude', 'Latitude']].to_numpy()

def _intermediate_points(df1, rad1: int, df2, rad2: int):
  int_points = []
  if rad1 >= rad2:
    for df1_row in df1:
      for df2_row in df2:
        loc_1 = (df1_row[0], df1_row[1])
        loc_2 = (df2_row[0], df2_row[1])
        center_dist = gcc.distance_between_points(loc_1, loc_2, unit='miles')
        #print(center_dist)

        if center_dist < (rad1 + rad2):
          int_point = gcc.intermediate_point(loc_1, loc_2, nom._intermediate_ratio(rad1, rad2))
          #print(int_point)
          int_points.append((int_point[1], int_point[0]))

  return int_points


int_points_arr = np.array(_intermediate_points(ikea_latlon_df, ikea_r, micro_latlon_df, micro_r))
# print(int_points_arr)
int_points_df = pd.DataFrame({'Latitude': int_points_arr[:, 0], 'Longitude': int_points_arr[:, 1]})

print("Intermediate Points - Ikea & Micro:", len(int_points_df.index))
int_points_df.head()
# print(int_points_df.to_clipboard())




print("Center ", nom._cent_points(int_points_arr))

Intermediate Points - Ikea & Micro: 93
Center  (40.29407402407976, -80.52373596724539)


In [966]:
cent_points_map = folium.Map(location=[centroid_points_dataframe.Latitude.mean(), centroid_points_dataframe.Longitude.mean()], zoom_start=4)

for point in centroid_points:
  folium.Circle([point[0], point[1]], nom._miles_to_meters(25)).add_to(cent_points_map)

cent_points_map